# Sentiment analysis 

## 1. Textblob-FR

Documentation: https://textblob.readthedocs.io/en/dev/

### Imports

In [ ]:
import sys
import os
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

### Création d'une fonction `get_sentiment`

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Analyser le sentiment d'une phrase

In [ ]:
# Choisir une année
year = 1900

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Liste pour stocker les phrases
selected_phrases = []

# Compteur pour les documents et les phrases
document_count = 0
phrase_count = 0

# Longueur minimale requise pour une phrase
min_phrase_length = 30

# Parcour des 10 premiers fichiers de la liste
for document in content_list[:10]:
    # Division du fichier en trois parties égales
    part_length = len(document) // 3
    
    # Phrases au début
    start_phrases = []
    current_phrase = ""
    for char in document[:part_length]:
        current_phrase += char
        if char in ['.', '!', '?']:
            if len(current_phrase) >= min_phrase_length:
                start_phrases.append(current_phrase)
                current_phrase = ""
    
    # Phrases au milieu
    middle_phrases = []
    current_phrase = ""
    for char in document[part_length:2*part_length]:
        current_phrase += char
        if char in ['.', '!', '?']:
            if len(current_phrase) >= min_phrase_length:
                middle_phrases.append(current_phrase)
                current_phrase = ""
    
    # Phrases à la fin
    end_phrases = []
    current_phrase = ""
    for char in document[2*part_length:]:
        current_phrase += char
        if char in ['.', '!', '?']:
            if len(current_phrase) >= min_phrase_length:
                end_phrases.append(current_phrase)
                current_phrase = ""
    
    # Sélection de 3 phrases au début, 3 au milieu et 4 à la fin
    selected_phrases.extend(start_phrases[:3])
    selected_phrases.extend(middle_phrases[:3])
    selected_phrases.extend(end_phrases[:4])
    
    # Mise à jour des compteurs
    document_count += 1
    phrase_count += len(start_phrases[:3]) + len(middle_phrases[:3]) + len(end_phrases[:4])
    
    if phrase_count >= 10:
        break


In [ ]:
 selected_phrases

In [ ]:
text = "\n".join(selected_phrases[:10])

In [ ]:
get_sentiment("Les journaux publient la dépêche suivante du camp de Frère, 31 : Les Boers ont établi un nouveau camp formé do 63 wagons.")

In [ ]:
get_sentiment(text)

## 2. Utilisation de transformers

Documentation: https://github.com/TheophileBlard/french-sentiment-analysis-with-bert

**!!** Si le code ne tourne pas sur votre machine, vous pouvez le tester directement sur Google Colab en utilisant [ce lien](https://colab.research.google.com/github/TheophileBlard/french-sentiment-analysis-with-bert/blob/master/colab/french_sentiment_analysis_with_bert.ipynb) **!!**

Le modèle peut également être testé en ligne sur [HuggingFace](https://huggingface.co/tblard/tf-allocine)

### Installation des librairies et imports

In [ ]:
!pip install tensorflow
!pip install sentencepiece
!pip install transformers
!pip install spacy-transformer

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

### Chargement du modèle

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

### Analyser le sentiment d'une phrase

In [ ]:
sentiment_analyser("Ce journal est vraiment super intéressant.")

In [ ]:
sentiment_analyser("Cette phrase est négative et je ne suis pas content !")